# 1. Otimizando os hiper-parâmetros do modelo: utilizando o *Grid Search*
* Uma vez definido os possíveis modelos que serão usados no projeto de *machine learning* é preciso ajustar os hiper-parâmetros desses modelos.
* Até o momento, o único modelo que foi visto que se faz necessário ajustar os hiper-parâmetros é o *Random Forest* para regressão (documentação do *Random Forest*: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html).
* Um hiper-parâmetro interessante para ser ajustado: 
    - *n_estimators*: quantidade de árvores na florest.
* É possível ir testando vários valores para o *n_estimators* manualmente, o que pode demorar muito. Sabendo disso, é mais interessante utilizar o *GridSearchCV* já fornecido pelo *scikit-learn*.
* O *GridSearchCV* realiza uma busca em grade (ou seja, testa todas as combinações dos hiper-parâmetros dados como entrada) e encontra a combinação de hiper-parâmetros que conseguiu o melhor *score* (documentação para o *GridSearchCV*: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).
* Os valores dos hiper-parâmetros que se deseja percorrer devem ser descritos em um dicionário e dados como entrada para o *GridSearchCV*.
* Ainda a função que calcula o *score* deve ser fornecida.

In [32]:
import numpy as np

params_grid = {"n_estimators": np.arange(5, 100, 5)}
params_grid

{'n_estimators': array([ 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85,
        90, 95])}

## 1.1 Exercício
1. Utilize o conjunto de dados *brazilian_houses_to_rent* e o modelo *RandomForestRegressor*.
2. Através da *GridSearchCV* encontre o melhor valor para o hiper-parâmetro *n_estimators*.
3. Para calcular o *score* utilize a função *mean_squared_error*.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder


from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, make_scorer

In [4]:
data = pd.read_csv("./houses_to_rent_v2.csv")

# tratamento de dados categóricos e preparação do dataframe para o modelo
city_feature = data[['city']].to_numpy()
onehot_encoder = OneHotEncoder().fit(city_feature)
cat_city = onehot_encoder.transform(city_feature).toarray()

categorial_features = data[["animal", "furniture"]].to_numpy()
ordinal_encoder = OrdinalEncoder().fit(categorial_features)
cat_data = ordinal_encoder.transform(categorial_features)

data_numeric = data.drop(['city', 'floor', 'animal', 'furniture', 'property tax (R$)', 'fire insurance (R$)', 'total (R$)'], axis = 1)

data_all = np.concatenate([data_numeric, cat_city, cat_data], axis=1)

x, y = np.delete(data_all, [5], axis=1), data_all[:,5]

In [6]:
scoring = make_scorer(mean_squared_error)

grid_search = GridSearchCV(RandomForestRegressor(), params_grid, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
grid_search.fit(x, y)

GridSearchCV(cv=10, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'n_estimators': array([ 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85,
       90, 95])},
             scoring='neg_mean_squared_error')

In [7]:
grid_search.best_params_

{'n_estimators': 80}

In [8]:
-grid_search.best_score_

4770176.6518340325

In [9]:
(-grid_search.cv_results_['mean_test_score']).min()

4770176.6518340325

In [10]:
cvres = grid_search.cv_results_

for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(-mean_score, params)

5647760.312296626 {'n_estimators': 5}
5102016.834431237 {'n_estimators': 10}
5062658.941633463 {'n_estimators': 15}
4921268.081108653 {'n_estimators': 20}
4953949.197783743 {'n_estimators': 25}
4859070.305600601 {'n_estimators': 30}
4849728.853787752 {'n_estimators': 35}
4858993.490738552 {'n_estimators': 40}
4792605.469268855 {'n_estimators': 45}
4816320.139811057 {'n_estimators': 50}
4797505.736705427 {'n_estimators': 55}
4795504.412882103 {'n_estimators': 60}
4812550.9011331955 {'n_estimators': 65}
4796811.410751867 {'n_estimators': 70}
4814451.349073672 {'n_estimators': 75}
4770176.6518340325 {'n_estimators': 80}
4781752.721042233 {'n_estimators': 85}
4780635.631284639 {'n_estimators': 90}
4783458.142247497 {'n_estimators': 95}


In [11]:
grid_search.best_estimator_

RandomForestRegressor(n_estimators=80)

# 2. Uma alternativa ao *Grid Search*: *Randomized Search*
* Para poucos combinações de valores o *Grid Search* realiza bem o seu trabalho.
* No entanto, quando se tem uma grande quantidade de valores e hiper-parâmetros a serem testados, o processo de validação cruzada pode ser muito demorado.
* Uma alternativa é utilizar o *Randomized Search* (*RandomizedSearchCV* no *scikit-learn*: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html).
* Ao contrário do *GridSearchCV*, o *RandomizedSearchCV* não avalia todas as possibilidades de combinações de valores, mas apenas uma parte das combinações obtidas de forma aleatória.
* O parâmetros *n_iter* define quantas combinações serão amostradas dos valores fornecidos nos hiper-parâmetros.

## 2.1 Exercício
1. Utilize o conjunto de dados *brazilian_houses_to_rent* e o modelo *RandomForestRegressor*.
2. Através da *RandomizedSearchCV* encontre o melhor valor para os hiper-parâmetros *n_estimators* e *max_features*.
3. Para calcular o *score* utilize a função *mean_squared_error*.

In [13]:
num_features = 11
param_grid = {
    "n_estimators": np.arange(5, 100, 5),
    "max_features": np.arange(2, num_features, 2)
}
param_grid

{'n_estimators': array([ 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85,
        90, 95]),
 'max_features': array([ 2,  4,  6,  8, 10])}

In [23]:
#grid_search = GridSearchCV(RandomForestRegressor(), params_grid, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
search = RandomizedSearchCV(RandomForestRegressor(), param_grid, scoring='neg_mean_squared_error', n_jobs=-1, cv=5)
search.fit(x,y)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
                   param_distributions={'max_features': array([ 2,  4,  6,  8, 10]),
                                        'n_estimators': array([ 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85,
       90, 95])},
                   scoring='neg_mean_squared_error')

In [24]:
search.best_estimator_

RandomForestRegressor(max_features=2, n_estimators=80)

In [25]:
search.best_params_

{'n_estimators': 80, 'max_features': 2}

In [27]:
-search.best_score_

4631952.110011336

In [30]:
(-search.cv_results_['mean_test_score']).min()

4631952.110011336

In [31]:
search.cv_results_

{'mean_fit_time': array([1.29334497, 1.25219402, 2.21405463, 5.08888083, 0.70130019,
        2.58004136, 6.23241239, 1.48912811, 1.96745205, 0.93272185]),
 'std_fit_time': array([0.32393785, 0.15580854, 0.37248656, 0.7906564 , 0.1166544 ,
        0.65043883, 1.16212085, 0.41995578, 0.38981984, 0.22805626]),
 'mean_score_time': array([0.10826597, 0.06222501, 0.07137656, 0.22029142, 0.04284515,
        0.15611162, 0.15884185, 0.07056031, 0.09240503, 0.02743192]),
 'std_score_time': array([0.02605527, 0.01721656, 0.02938346, 0.04744793, 0.0096103 ,
        0.04860404, 0.03321149, 0.02100724, 0.0277288 , 0.00879931]),
 'param_n_estimators': masked_array(data=[40, 30, 40, 85, 25, 80, 90, 40, 45, 15],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_max_features': masked_array(data=[2, 4, 8, 8, 2, 2, 10, 4, 6, 10],
              mask=[False, False, False, False, False, Fa